# London Case Study

#### Work created by Noah Scanlon

## Imports

In [1]:
from sqlalchemy import create_engine
import pandas as pd

## Establish Connection to SQLite DB

In [2]:
database = 'sqlite:///sqlite_db_pythonsqlite.db'

In [75]:
engine = create_engine(database)

In [32]:
cnx = engine.connect()

## country_club Tables

In [135]:
engine.table_names()

['Bookings', 'Facilities', 'Members']

In [48]:
facilities_df = pd.read_sql_table('Facilities', cnx)
bookings_df = pd.read_sql_table('Bookings', cnx)
members_df = pd.read_sql_table('Members', cnx);

In [49]:
facilities_df

,facid,name,membercost,guestcost,initialoutlay,monthlymaintenance
0,0,Tennis Court 1,5.0,25.0,10000,200
1,1,Tennis Court 2,5.0,25.0,8000,200
2,2,Badminton Court,0.0,15.5,4000,50
3,3,Table Tennis,0.0,5.0,320,10
4,4,Massage Room 1,9.9,80.0,4000,3000
5,5,Massage Room 2,9.9,80.0,4000,3000
6,6,Squash Court,3.5,17.5,5000,80
7,7,Snooker Table,0.0,5.0,450,15
8,8,Pool Table,0.0,5.0,400,15


In [51]:
bookings_df.head()

,bookid,facid,memid,starttime,slots
0,0,3,1,2012-07-03 11:00:00,2
1,1,4,1,2012-07-03 08:00:00,2
2,2,6,0,2012-07-03 18:00:00,2
3,3,7,1,2012-07-03 19:00:00,2
4,4,8,1,2012-07-03 10:00:00,1


In [52]:
members_df.head()

,memid,surname,firstname,address,zipcode,telephone,recommendedby,joindate
0,0,GUEST,GUEST,GUEST,0,(000) 000-0000,,2012-07-01 00:00:00
1,1,Smith,Darren,"8 Bloomsbury Close, Boston",4321,555-555-5555,,2012-07-02 12:02:05
2,2,Smith,Tracy,"8 Bloomsbury Close, New York",4321,555-555-5555,,2012-07-02 12:08:23
3,3,Rownam,Tim,"23 Highway Way, Boston",23423,(844) 693-0723,,2012-07-03 09:32:15
4,4,Joplette,Janice,"20 Crossing Road, New York",234,(833) 942-4710,1,2012-07-03 10:25:05


## Queries

### Q10

In [99]:
q10_query = '''
            SELECT
                f.name AS facility_name,
                SUM(CASE WHEN b.memid = 0 THEN guestcost
                    ELSE membercost END) AS revenue
            FROM Bookings AS b
            JOIN Facilities AS f
                ON b.facid = f.facid
            JOIN Members AS m
                ON b.memid = m.memid
            GROUP BY b.facid
            HAVING revenue < 1000
            ORDER BY revenue DESC;
            '''

In [181]:
q10_df = pd.read_sql_query(q10_query, cnx)

#### Q10 Solution

In [182]:
q10_df

,facility_name,revenue
0,Badminton Court,604.5
1,Pool Table,265.0
2,Snooker Table,115.0
3,Table Tennis,90.0


### Q11

In [127]:
q11_query = ''' 
            SELECT m1.firstname || ' ' || m1.surname AS member_name, m2.firstname || ' ' || m2.surname AS recommender_name
            FROM Members AS m1
            JOIN Members AS m2
                ON m1.recommendedby = m2.memid
            WHERE m1.recommendedby = m2.memid
            ORDER BY m1.surname, m1.firstname;
            '''

In [179]:
q11_df = pd.read_sql_query(q11_query, cnx)

#### Q11 Solution

In [180]:
q11_df

,member_name,recommender_name
0,Florence Bader,Ponder Stibbons
1,Anne Baker,Ponder Stibbons
2,Timothy Baker,Jemima Farrell
3,Tim Boothe,Tim Rownam
4,Gerald Butters,Darren Smith
5,Joan Coplin,Timothy Baker
6,Erica Crumpet,Tracy Smith
7,Nancy Dare,Janice Joplette
8,Matthew Genting,Gerald Butters
9,John Hunt,Millicent Purview


### Q12

In [167]:
q12_query = ''' 
            SELECT f.name, b.memid, m.firstname || ' ' || m.surname AS member_name, COUNT(b.memid) AS COUNT_member_usage
            FROM Bookings AS b
            JOIN Facilities AS f
                ON b.facid = f.facid
            JOIN Members AS m
                ON b.memid = m.memid
            WHERE b.memid != 0
            GROUP BY b.facid, b.memid;
            '''

In [177]:
q12_df = pd.read_sql_query(q12_query, cnx)

#### Q12 Solution

In [178]:
q12_df

,name,memid,member_name,COUNT_member_usage
0,Tennis Court 1,2,Tracy Smith,30
1,Tennis Court 1,3,Tim Rownam,6
2,Tennis Court 1,4,Janice Joplette,19
3,Tennis Court 1,5,Gerald Butters,57
4,Tennis Court 1,6,Burton Tracy,31
...,...,...,...,...
197,Pool Table,27,Henrietta Rumney,3
198,Pool Table,28,David Farrell,25
199,Pool Table,29,Henry Worthington-Smyth,33
200,Pool Table,30,Millicent Purview,5


### Q13

In [192]:
q13_query = ''' 
            SELECT f.name AS facility_name, strftime('%m', starttime) AS month, COUNT(b.memid) AS COUNT_usage
            FROM Bookings AS b
            JOIN Facilities AS f
                ON b.facid = f.facid
            JOIN Members AS m
                ON b.memid = m.memid
            WHERE b.memid != 0
            GROUP BY b.facid, month;
            '''

In [193]:
q13_df = pd.read_sql_query(q13_query, cnx)

#### Q13 Solution

In [194]:
q13_df

,facility_name,month,COUNT_usage
0,Tennis Court 1,07,65
1,Tennis Court 1,08,111
2,Tennis Court 1,09,132
3,Tennis Court 2,07,41
4,Tennis Court 2,08,109
5,Tennis Court 2,09,126
6,Badminton Court,07,51
7,Badminton Court,08,132
8,Badminton Court,09,161
9,Table Tennis,07,48
